presentation notes:

history:

Git was started by the same guy who started Linux kernel. But of course, as both software are open source, there have been many developers who contributed their work over the years. However, as the guy who started it, Linus Torvald, has always been the main driving force behind it.

Linux kernel, the basis of the many distributions of Linux platforms that we have today, is an incredibly complex project. And it was using a paid version control system at the time. However, as that particular version control up'ed its price and the project was growing too big for it to handle, Linus cut the ties and started looking for alternatives.

However, he couldn't find anything on the market that suits his needs. He needed three things. Distributed developments, supports many developers at a time. Scale, needs to be able to handle large amount of code, many different versions. Speed, it needs to be able to do its job fast. And low cost, preferably free.

As he was looking for the tool, the linux community can't wait. There's ongoing development, and the code base has grown to such a size, and there have been so many different versions coming from so many different developers. He could no longer handle it with the current set of tools.

So he stopped. he stopped the development of the project and said, we can no longer continue this project until a capable tool is found. Or made. So that's how git came to life.

what it is:

so what is git? in simple terms, if you take away all the fluff, it is a tree of commits in order. It's tree, not a list, since it supports branching. Then what is a commit?

what is a commit?

A commit is a collection of changes. And there are four types of changes that can be included in any commit. Adding a file, removing a file, renaming or moving a file, and modifying a file. inside a file change, added lines are denoted by ++, deleted lines are denoted by --.

Quick comparison between git and clearcase:
It is offline. Yes. offline. Your repo on your local drive, it is considered a complete package. Yes, if you reformat your computer, you're gonna lose everything. Hence why people sync their repos to the cloud services. But, as it stands, without the remote point, you have access to all the version control functions on your local computer.
